# Scraping Transfermarkt

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Function that scrapes a list of Team Webpage Links

In [10]:
#top_league(n) returns a list of links to the webpages of every club within the top n European leagues.
#i.e. Passing 1 as an argument will return links to the no.1 league in Europe- England. Passing 2 as an argument will return links of clubs in the 2 best leagues.
def top_league(n):

    # headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    pageTree = requests.get(page)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    links = pageSoup.find_all("tr")

    site = 'https://www.transfermarkt.co.uk'
    s23_string = "/plus/?saison_id=2023"

    top_leagues = []
    for a in list(range(2,((n*2)+1),2)):
        top_leagues.append((site+(str(links[a]).split('href="',5)[2].split('"')[0])+s23_string))

    teams = []

    for a in top_leagues:

        page = a
        pageTree = requests.get(page)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        links2 = pageSoup.find_all("a",{"class":"vereinprofil_tooltip"})
        site = 'https://www.transfermarkt.co.uk'
        s22_string = "/plus/?saison_id=2022"

        if a=="https://www.transfermarkt.co.uk/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2020" or a=="https://www.transfermarkt.co.uk/liga-portugal-bwin/startseite/wettbewerb/PO1/plus/?saison_id=2020" or a=="'https://www.transfermarkt.co.uk/eredivisie/startseite/wettbewerb/NL1/plus/?saison_id=2020'":
            for b in list(range(0,(18*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s22_string)
        elif a== 'https://www.transfermarkt.co.uk/premier-liga/startseite/wettbewerb/RU1/plus/?saison_id=2020':
            for b in list(range(0,(16*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s22_string) 
        else:
            for b in list(range(0,(20*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s22_string)
    return teams

## Function that scrapes a list of Individual Player Webpage Links

In [17]:
from urllib.parse import urlencode
#top_league_player(n) returns a list of links to the individual profile webpage for every player participating in Europe's top n number of leagues.
def top_league_player(n):

    # headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    params = {'api_key': '055af10c5c5aafd1939eb8e367a32334', 'url': page}
    pageTree = requests.get(page, params=urlencode(params))
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
    print(pageSoup)

    links = pageSoup.find_all("tr")
    print(links)

    site = 'https://www.transfermarkt.co.uk'
    s23_string = "/plus/?saison_id=2023"

    top_leagues = []
    for a in list(range(2,((n*2)+1),2)):
        top_leagues.append((site+(str(links[a]).split('href="',5)[2].split('"')[0])+s23_string))

    teams = []

    for a in top_leagues:

        page = a
        pageTree = requests.get(page)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        links2 = pageSoup.find_all("a",{"class":"vereinprofil_tooltip"})
        site = 'https://www.transfermarkt.co.uk'
        s22_string = "/plus/?saison_id=2022"

        if a=="https://www.transfermarkt.co.uk/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2020" or a=="https://www.transfermarkt.co.uk/liga-portugal-bwin/startseite/wettbewerb/PO1/plus/?saison_id=2020" or a=="'https://www.transfermarkt.co.uk/eredivisie/startseite/wettbewerb/NL1/plus/?saison_id=2020'":
            for b in list(range(0,(18*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s22_string)
        elif a== 'https://www.transfermarkt.co.uk/premier-liga/startseite/wettbewerb/RU1/plus/?saison_id=2020':
            for b in list(range(0,(16*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s22_string) 
        else:
            for b in list(range(0,(20*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s22_string)
    
    player_links = []
    for a in teams:
        page = a
        pageTree = requests.get(page)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        site = 'https://www.transfermarkt.co.uk'

        links = pageSoup.find_all("a",{"class":"spielprofil_tooltip"})

        for b in list(range(0,len(links),2)):
            player_links.append(site+(str(links[b]).split('href="')[1].split('"')[0]))
    
    return player_links
    

## Scraping Player Contract Length Data

Contract dataframe: contract scrape from individual player pages. #Warning: IT MAY TAKE HOURS TO RUN but it WORKS!!!

In [18]:
#iterating through every link in the list provided to it and then returns a dataframe of player data from the given webpages.
#top_league_player(5) returns a list of links to the individual profile webpage for every player participating in Europe's top 5 leagues.

site = 'https://www.transfermarkt.co.uk'
PlayerList = []
ContractList = []

# headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

for a in top_league_player(5):

    page = a
    pageTree = requests.get(page)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    Player = pageSoup.find_all("meta",{"property":"og:title"})
    Contract = pageSoup.find_all("span",{"class":"dataValue"})

    PlayerList.append(str(Player).split('content="')[1].split(" - ")[0])
    try:
        ContractList.append((int(str(Contract[-1]).split(", ")[1].split("</span>")[0])-2021))
    except IndexError:
        ContractList.append("fail")

contract_df = pd.DataFrame({"Player":PlayerList,"Contract Years Left":ContractList})

<html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<center><h1>404 Not Found</h1></center>
<hr/><center>nginx</center>
</body>
</html>

[]


IndexError: list index out of range

In [ ]:
#Observing what our contract df looks like
contract_df

Player Contract Years Left
0               Alisson                   6
1     Caoimhin Kelleher                   5
2          Loris Karius                   1
3                Adrián                   1
4       Virgil van Dijk                   4
...                 ...                 ...
3168    Matthieu Dreyer                   2
3169       Théo Le Bris                fail
3170     Redwan Bourlès                fail
3171        Enzo Le Fée                   3
3172  Pierre-Yves Hamel                   1

[3173 rows x 2 columns]

## Function that Scrapes and builds a dataframe of all Players' data

In [ ]:
#This function works when providing links of individual club pages.
#It iterates through every link in the list provided to it and then returns a dataframe of player data from the given webpages.
def build_df(links):

    PlayersList = []
    AgeList=[]
    NationList=[]
    ValuesList = []
    PositionsList=[]
    ClubList = []

    for a in links:

        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Players = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
        Age = pageSoup.find_all("td", {"class": "zentriert"})
        Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})
        Positions = pageSoup.find_all("td", {"class": ["zentriert rueckennummer bg_Torwart","zentriert rueckennummer bg_Abwehr","zentriert rueckennummer bg_Mittelfeld","zentriert rueckennummer bg_Sturm"]})
        Nationality = pageSoup.find_all("td", {"class": "zentriert"})
        Club = pageSoup.find_all("meta")

        for i in range(0,len(Values)):
            ValuesList.append(Values[i].text)

        for i in range(0,len(Players)):
            PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

        for i in range(0,len(Positions)):
            PositionsList.append(str(Positions[i]).split('title="',1)[1].split('"><div')[0])

        for i in range(1,(len(Players)*3),3):
            AgeList.append(str(Age[i]).split("(",1)[1].split(")",1)[0])

        for i in range(2,(len(Players)*3),3):
            NationList.append(str(Nationality[i]).split('title="',1)[1].split('"/',1)[0])

        for i in range(0,len(Players)):
            ClubList.append(str(Club[6]).split('content="')[1].split(",")[0])
            

    #Initial uncleaned dataframe initiated
    df= pd.DataFrame({"Players":PlayersList,"Club":ClubList,"Position":PositionsList, "Age":AgeList, "Nationality":NationList,"Values":ValuesList,})

    #Missing Transfer Values were stored as '\xa0'. The line below replaces them as None values
    df.loc[(df.Values == '\xa0'),'Values']= None

    #Dataframe without missing transfer values
    df_droppednull = df.dropna()

    #Converting the Transfer Values data to float            
    transfer_values = [a[1:len(a)-2] for a in df_droppednull['Values']]

    cleaned_values=[]
    for a in list(range(len(transfer_values))):
        if transfer_values[a].endswith('m'):
            cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-1])*1000000)
        elif transfer_values[a].endswith('Th.'):
            cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-3])*1000)
        else:
            cleaned_values.append(float(a))

    #Constructing the Final Dataframe   
    final_df= pd.DataFrame({"Player":df_droppednull['Players'],"Club":df_droppednull['Club'],
                    "Age":df_droppednull['Age'],"Position":df_droppednull['Position'],
                    "Nation":df_droppednull['Nationality'],"Value":cleaned_values})

    #df ranked by transfer value
    ranked_df = final_df.sort_values('Value',ascending=False)
    
    return ranked_df

Building the dataframe of players from every Club Page

In [ ]:
#Building a dataframe by scraping information of the 98 teams from Europe's top 5 leagues - i.e. England, Spain, Italy, Germany & France
first_df = build_df(top_league(5))

### Seeing what our scraped dataset looks like

In [ ]:
#Seeing what our dataframe looks like
first_df

Player                 Club Age    Position   Nation  \
2132        Kylian Mbappé  Paris Saint-Germain  22      attack   France   
1637       Erling Haaland    Borussia Dortmund  21      attack   Norway   
98             Harry Kane    Tottenham Hotspur  28      attack  England   
121          Jadon Sancho    Manchester United  21      attack  England   
23          Mohamed Salah         Liverpool FC  29      attack    Egypt   
...                   ...                  ...  ..         ...      ...   
1392    Michele Laraspata    Parma Calcio 1913  20    Defender    Italy   
1479   Alessandro Berardi        Hellas Verona  30  Goalkeeper    Italy   
2473         Mamadou Mbow          Stade Reims  21    Defender  Senegal   
1397  Alessandro Martella    Parma Calcio 1913  20    Defender    Italy   
1931     Matthias Köbbing           1. FC Köln  24  Goalkeeper  Germany   

            Value  
2132  144000000.0  
1637  117000000.0  
98    108000000.0  
121    90000000.0  
23     90000000.0  
...           ...  
1392      45000.0  
1479      45000.0  
2473      45000.0  
1397      23000.0  
1931       9000.0  

[2635 rows x 6 columns]

## Merging the contract length data onto the above dataframe

In [ ]:
#merging our dataframe with contract information
merged_df = pd.merge(first_df, contract_df, on='Player')

## Adding a League Column

In [ ]:
#Grouping Clubs by individual Leagues and making a separate "League" column.

England = ['Manchester City','Liverpool FC','Crystal Palace','Chelsea FC','Fulham FC','Brighton &amp; Hove Albion','West Bromwich Albion','Newcastle United','Leeds United','Aston Villa','Sheffield United','Manchester United','West Ham United','Leicester City','Tottenham Hotspur','Burnley FC','Southampton FC','Arsenal FC','Wolverhampton Wanderers','Everton FC']
Spain = ['SD Eibar','CA Osasuna','Athletic Bilbao','Real Betis Balompié','Valencia CF','Celta de Vigo','Levante UD','Cádiz CF','FC Barcelona','Villarreal CF','SD Huesca','Sevilla FC','Getafe CF','Deportivo Alavés','Real Valladolid CF','Granada CF','Real Madrid','Elche CF','Atlético de Madrid','Real Sociedad']
Italy = ['US Sassuolo','AC Milan','Juventus FC','Udinese Calcio','Genoa CFC','Cagliari Calcio','Inter Milan','FC Crotone','Hellas Verona','Benevento Calcio','Parma Calcio 1913','SSC Napoli','AS Roma','Bologna FC 1909','Torino FC','UC Sampdoria','SS Lazio','ACF Fiorentina','Atalanta BC','Spezia Calcio']
Germany = ['Bayern Munich','Hertha BSC','Borussia Dortmund','VfB Stuttgart','1.FSV Mainz 05','Arminia Bielefeld','Bayer 04 Leverkusen','SV Werder Bremen','1. FC Köln','FC Augsburg','VfL Wolfsburg','TSG 1899 Hoffenheim','RB Leipzig','SC Freiburg','1.FC Union Berlin','FC Schalke 04','Borussia Mönchengladbach','Eintracht Frankfurt']
France = ['AS Monaco','Montpellier HSC','FC Girondins Bordeaux','Paris Saint-Germain','Olympique Marseille','OGC Nice','Nîmes Olympique','FC Nantes','Stade Reims','FC Lorient','Stade Brestois 29','LOSC Lille','Stade Rennais FC','Dijon FCO','RC Strasbourg Alsace','RC Lens','AS Saint-Étienne','FC Metz','Olympique Lyon','SCO Angers']

LeagueList = []
for a in merged_df['Club']:
    if a in England:
        LeagueList.append('Premier League')
    elif a in Spain:
        LeagueList.append('La Liga')
    elif a in Italy:
        LeagueList.append('Serie A')
    elif a in Germany:
        LeagueList.append('Bundesliga')
    elif a in France:
        LeagueList.append('Ligue 1')
    else:
        LeagueList.append("UNKNOWN")
        
merged_df['League'] = LeagueList

In [ ]:
#looking at our dataframe with contract information appended.
merged_df

Player                 Club Age    Position   Nation  \
0           Kylian Mbappé  Paris Saint-Germain  22      attack   France   
1          Erling Haaland    Borussia Dortmund  21      attack   Norway   
2          Erling Haaland    Borussia Dortmund  21      attack   Norway   
3              Harry Kane    Tottenham Hotspur  28      attack  England   
4            Jadon Sancho    Manchester United  21      attack  England   
...                   ...                  ...  ..         ...      ...   
3100    Michele Laraspata    Parma Calcio 1913  20    Defender    Italy   
3101   Alessandro Berardi        Hellas Verona  30  Goalkeeper    Italy   
3102         Mamadou Mbow          Stade Reims  21    Defender  Senegal   
3103  Alessandro Martella    Parma Calcio 1913  20    Defender    Italy   
3104     Matthias Köbbing           1. FC Köln  24  Goalkeeper  Germany   

            Value Contract Years Left          League  
0     144000000.0                   1         Ligue 1  
1     117000000.0                   3      Bundesliga  
2     117000000.0                   3      Bundesliga  
3     108000000.0                   3  Premier League  
4      90000000.0                   5  Premier League  
...           ...                 ...             ...  
3100      45000.0                   3         Serie A  
3101      45000.0                   2         Serie A  
3102      45000.0                   2         Ligue 1  
3103      23000.0                   3         Serie A  
3104       9000.0                   1      Bundesliga  

[3105 rows x 8 columns]

In [ ]:
#There are some duplicate rows
merged_df.drop_duplicates(keep="first",inplace=True)

In [ ]:
merged_df

Player                 Club Age    Position   Nation  \
0           Kylian Mbappé  Paris Saint-Germain  22      attack   France   
1          Erling Haaland    Borussia Dortmund  21      attack   Norway   
3              Harry Kane    Tottenham Hotspur  28      attack  England   
4            Jadon Sancho    Manchester United  21      attack  England   
5           Mohamed Salah         Liverpool FC  29      attack    Egypt   
...                   ...                  ...  ..         ...      ...   
3100    Michele Laraspata    Parma Calcio 1913  20    Defender    Italy   
3101   Alessandro Berardi        Hellas Verona  30  Goalkeeper    Italy   
3102         Mamadou Mbow          Stade Reims  21    Defender  Senegal   
3103  Alessandro Martella    Parma Calcio 1913  20    Defender    Italy   
3104     Matthias Köbbing           1. FC Köln  24  Goalkeeper  Germany   

            Value Contract Years Left          League  
0     144000000.0                   1         Ligue 1  
1     117000000.0                   3      Bundesliga  
3     108000000.0                   3  Premier League  
4      90000000.0                   5  Premier League  
5      90000000.0                   2  Premier League  
...           ...                 ...             ...  
3100      45000.0                   3         Serie A  
3101      45000.0                   2         Serie A  
3102      45000.0                   2         Ligue 1  
3103      23000.0                   3         Serie A  
3104       9000.0                   1      Bundesliga  

[2643 rows x 8 columns]

## Exporting the Dataset

In [ ]:
# Install gitpath below if not already available on your environment.
#! pip install git+https://github.com/maxnoe/python-gitpath
import gitpath

#Getting the file path for the data file.
file_path = str(gitpath.root()) + '/data/transfermarkt_scrape_latest.xlsx'

In [ ]:
#Exporting data to .csv format in the repo's data folder.
merged_df.to_excel(file_path,index=False)